# Day 2 - Lab 2: Documenting Key Decisions with ADRs

**Objective:** Use an LLM as a research assistant to compare technical options and synthesize the findings into a formal, version-controlled Architectural Decision Record (ADR).

**Estimated Time:** 60 minutes

**Introduction:**
Great architectural decisions are based on research and trade-offs. A critical practice for healthy, long-lived projects is documenting *why* these decisions were made. In this lab, you will use an LLM to research a key technical choice for our application and then generate a formal ADR to record that decision for the future.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We'll start by ensuring our environment is ready and adding the standard pathing solution to reliably import our `utils.py` helper.

**Model Selection:**
For research and synthesis tasks, models with large context windows and strong reasoning abilities are ideal. `gpt-4.1`, `gemini-2.5-pro`, or `meta-llama/Llama-3.3-70B-Instruct` would be excellent choices.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the ADR template.
- `save_artifact()`: To save the generated ADR template and the final ADR.

In [37]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

2025-10-29 09:52:58,290 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): The ADR Template

**Task:** A good ADR follows a consistent format. Your first task is to prompt an LLM to generate a clean, reusable ADR template in markdown.

**Instructions:**
1.  Write a prompt that asks the LLM to generate a markdown template for an Architectural Decision Record.
2.  The template should include sections for: `Title`, `Status` (e.g., Proposed, Accepted, Deprecated), `Context` (the problem or forces at play), `Decision` (the chosen solution), and `Consequences` (the positive and negative results of the decision).
3.  Save the generated template to `templates/adr_template.md`.

In [38]:
adr_template_prompt = """
Create a comprehensive markdown template for an Architecture Decision Record (ADR) that follows industry best practices and includes the following sections:

**Required Sections:**
1. **Title** - A clear, descriptive title using the format "ADR-[Number]: [Brief Decision Description]"
2. **Status** - Current state (Proposed, Accepted, Superseded, Deprecated, Rejected)
3. **Context** - The technical and business forces, constraints, and requirements driving this decision
4. **Decision** - The specific choice made, including key implementation details
5. **Consequences** - Both positive and negative outcomes, trade-offs, and implications

**Additional Requirements:**
- Include metadata section with Date, Author(s), and Stakeholders
- Provide clear placeholder text with examples for each section
- Use proper markdown formatting with headers, lists, and emphasis
- Include guidance comments to help users fill out each section effectively
- Follow the standard ADR numbering convention (ADR-001, ADR-002, etc.)

**Output Format:**
- Use consistent markdown syntax with proper heading hierarchy
- Structure the template to be both human-readable and version-control friendly

The template should be professional, comprehensive, and suitable for enterprise software development teams.

Return ONLY markdown. Do not include any explanations or additional text outside the markdown format. Avoid using code blocks. Do not fill in any sections or example, only provide a template with placeholders.
"""

print("--- Generating ADR Template ---")
adr_template_content = get_completion(adr_template_prompt, client, model_name, api_provider)
print(adr_template_content)

# Save the artifact
if adr_template_content:
    save_artifact(adr_template_content, "templates/adr_template.md", overwrite=True)  # rewrite file on rerun of program.

--- Generating ADR Template ---
# ADR-[Number]: [Brief Decision Description]

*   **Date**: `YYYY-MM-DD`
*   **Author(s)**: `[List of author names and/or emails]`
*   **Stakeholders**: `[List of key stakeholders, e.g., team leads, product owners, architects]`

## Status

`[Proposed | Accepted | Rejected | Deprecated | Superseded by ADR-XXX]`

*This section should be updated as the ADR moves through its lifecycle. See below for status definitions.*
*   **Proposed**: This decision is under review.
*   **Accepted**: The decision has been approved and is the current standard.
*   **Rejected**: The decision was proposed but not approved.
*   **Deprecated**: The decision was previously accepted but is no longer recommended. A new ADR may or may not be in place.
*   **Superseded by ADR-XXX**: The decision has been replaced by a new decision, documented in the linked ADR.

## Context

*This section describes the problem, the forces at play, and the constraints that influence the decision. It s

### Challenge 2 (Intermediate): AI-Assisted Research

**Task:** Use the LLM to perform unbiased research on a key technical decision for our project: choosing a database for semantic search.

**Instructions:**
1.  Write a prompt instructing the LLM to perform a technical comparison.
2.  Ask it to compare and contrast two technical options: **"Using PostgreSQL with the `pgvector` extension"** versus **"Using a specialized vector database like ChromaDB or FAISS"**.
3.  The prompt should ask for a balanced view for the specific use case of our new hire onboarding tool.
4.  Store the output in a variable for the next step.

> **Tip:** To get a balanced comparison, explicitly ask the LLM to 'act as an unbiased research assistant' and to list the 'pros and cons for each approach.' This prevents the model from simply recommending the more popular option and encourages a more critical analysis.

In [39]:
pdr_document = load_artifact("day1_prd_2025-10-28_14-03-52.md")

db_research_prompt = f"""
Act as an unbiased technology researcher. Research and compare the following database options for a new software project. Provide pros and cons for each option, considering factors such as scalability, performance, ease of use, cost, and community support.
The database options to research are:
1. PostgreSQL with the pgvector extension
2. Using a specialized vector database like ChromaDB or FAISS

Compare the two options within the context of the following project requirements documentation:
<pdr_document>
{pdr_document}
</pdr_document>

Use web search to gather the most up-to-date information on each database option, including recent benchmarks, user reviews, and case studies.
Present your findings in a clear, structured format, such as a table or bullet points, to help stakeholders make an informed decision.
Only provide results from this question. Only include well-formated markdown in your response. Do not include any explanations or additional text outside the markdown format.
"""

print("--- Researching Database Options ---")
db_research_output = get_completion(db_research_prompt, client, model_name, api_provider)
print(db_research_output)

--- Researching Database Options ---
### **Unbiased Technology Research Report: Database Options for the Ascend Onboarding Platform**

**Date:** October 26, 2023
**Prepared By:** Unbiased Technology Researcher
**Subject:** Comparison of PostgreSQL with `pgvector` vs. Specialized Vector Databases for the Ascend Onboarding Platform

---

### **1. Executive Summary**

This report analyzes two primary database approaches for the Ascend Onboarding Platform, focusing on the requirements outlined in the Product Requirements Document (PRD v1.0). The core functionality requiring vector search capabilities is the "searchable resource library" (Story 1.4) and future AI-powered personalization (Section 7.2).

1.  **PostgreSQL with `pgvector`:** This approach integrates vector search capabilities directly into a mature, full-featured relational database. It offers significant advantages in architectural simplicity, data consistency, and reduced operational overhead, as all application data (user pr

### Challenge 3 (Advanced): Synthesizing the ADR

**Task:** Provide the LLM with your research from the previous step and have it formally document the decision.

**Instructions:**
1.  Load the `adr_template.md` you created in the first challenge.
2.  Create a new prompt instructing the LLM to act as a Staff Engineer.
3.  Provide the `db_research_output` as context.
4.  Instruct the LLM to populate the ADR template, formally documenting the decision to **use PostgreSQL with pgvector** and justifying the choice based on the synthesized pros and cons.
5.  Save the final, completed ADR as `artifacts/adr_001_database_choice.md`.

In [40]:
adr_template = load_artifact("templates/adr_template.md")

synthesis_prompt = f"""
Act as a Staff Software Engineer.

Using the following Architecture Decision Record (ADR) template and the database research findings, synthesize a complete ADR documenting the decision to use PostgreSQL with pgvector for the project.

Based on the research findings, formally document the decision to **use PostgreSQL with pgvector** and justify this choice based on the pros and cons.

## ADR Template
{adr_template}

## Database Research Findings
{db_research_output}

Please fill in all sections of the ADR, including Title, Status, Context, Decision, Consequences, and Metadata (Date, Author(s), Stakeholders). The decision should clearly state why PostgreSQL with pgvector should be chosen and provide justification based on the research findings.
Remove any placeholder text from the ADR template and replace it with relevant content.
Ensure the ADR is professional, clear, and suitable for enterprise software development. Only provide the markdown content of the completed ADR without any additional explanations or text.
"""

print("--- Synthesizing Final ADR ---")
if adr_template and 'db_research_output' in locals() and db_research_output:
    final_adr = get_completion(synthesis_prompt, client, model_name, api_provider)
    print(final_adr)
    save_artifact(final_adr, "artifacts/adr_001_database_choice.md", overwrite=True)  # rewrite file on rerun of program.
else:
    print("Skipping ADR synthesis because template or research is missing.")

--- Synthesizing Final ADR ---
# ADR-001: Choose PostgreSQL with pgvector for Primary Data and Vector Storage

*   **Date**: 2023-10-30
*   **Author(s)**: Staff Software Engineer, Ascend Platform Team
*   **Stakeholders**: Head of Engineering, Ascend Product Owner, Lead Platform Architect

## Status

Accepted

*This section should be updated as the ADR moves through its lifecycle. See below for status definitions.*
*   **Proposed**: This decision is under review.
*   **Accepted**: The decision has been approved and is the current standard.
*   **Rejected**: The decision was proposed but not approved.
*   **Deprecated**: The decision was previously accepted but is no longer recommended. A new ADR may or may not be in place.
*   **Superseded by ADR-XXX**: The decision has been replaced by a new decision, documented in the linked ADR.

## Context

*This section describes the problem, the forces at play, and the constraints that influence the decision. It should be detailed enough for a fu

## Lab Conclusion

Well done! You have used an LLM to automate a complex but critical part of the architectural process. You leveraged its vast knowledge base for research and then used it again for synthesis, turning raw analysis into a formal, structured document. This `adr_001_database_choice.md` file now serves as a permanent, valuable record for anyone who works on this project in the future.

> **Key Takeaway:** The pattern of **Research -> Synthesize -> Format** is a powerful workflow. You can use an LLM to gather unstructured information and then use it again to pour that information into a structured template, creating high-quality, consistent documentation with minimal effort.